# Merging of Tatoeba Datasets on the Local Machine

The purpose of this notebook is to establish a baseline for the kind of coding and time duration needed to undertake similar merges to what is being accomplished in the MRjob file (mapreduce.py). This code stops short of the final steps, that require producing just the final paraphrases in one language, rather than bidirectional translation pairs. The complexity of those omitted steps is one of the reasons I chose the MapReduce framework in the first place!

In [28]:
import pandas as pd
from tqdm import tqdm
import pickle
import time

In [47]:
pd.set_option('display.max_colwidth', -1)

First, we consider the time requirements for loading the data. This corresponds roughly to  

In [32]:
t0 = time.time()
sent_details = pd.read_csv('sentences.csv', 
                           sep='\t', header=None, error_bad_lines=False)
sent_details = sent_details.rename(columns={0:'Sentence id', 1:'Lang',  2:'Text'})
sent_details.dropna(subset=['Lang'], inplace=True)
links = pd.read_csv('links.csv', sep='\t', header=None, error_bad_lines=False)
links = links.rename(columns={0:'Sentence id',1:'Translation id'})

print("Time taken to load the two tables is {} seconds". format(time.time()-t0))

Time taken to load the two tables is 84.02482175827026 seconds


This implies that for our parallelized code to offer any computational benefits relative to serial code on local machines, it must take under 84 seconds to complete the first step of the Mapper (the splitting of rows and merging thereof based on the translation ID)

### Testing Unicode Complications

In [4]:
#Test what happens with splittling on TSV
# https://stackoverflow.com/questions/13992971/reading-and-parsing-a-tsv-file-then-manipulating-it-for-saving-as-csv-efficie
# Will need the following as support: 
# https://stackoverflow.com/questions/21955868/python-map-reduce-simple-wordcount-in-cyrillic-text
enc = 'utf-8'
with open('sentences_tsv_sample.csv','r', encoding=enc) as tsv:
    AoA = [line.strip().split('\t') for line in tsv]

In [5]:
print(AoA)

[['"1', 'cmn', '我們試試看！",'], ['"2', 'cmn', '我该去睡觉了。",'], ['"3', 'cmn', '你在干什麼啊？",'], ['"4', 'cmn', '這是什麼啊？",'], ['"5', 'cmn', '今天是６月１８号，也是Muiriel的生日！",'], ['"6', 'cmn', '生日快乐，Muiriel！",'], ['"7', 'cmn', 'Muiriel现在20岁了。",'], ['"8', 'cmn', '密码是""Muiriel""。",'], ['"9', 'cmn', '我很快就會回來。",'], ['"10', 'cmn', '我不知道。",'], ['"11', 'cmn', '我不知道應該說什麼才好。",'], ['"12', 'cmn', '這個永遠完不了了。",'], ['"13', 'cmn', '我只是不知道應該說什麼而已……",'], ['"14', 'cmn', '那是一隻有惡意的兔子。",'], ['"15', 'cmn', '我以前在山里。",'], ['"16', 'cmn', '那是一张近照吗？",'], ['"17', 'cmn', '我不知道我有沒有時間。",'], ['"18', 'cmn', '剛才我的麥克風沒起作用，不知道為什麼。",'], ['"19', 'cmn', '到了最後，大家一定要靠自己學習。",'], ['"20', 'cmn', '世界上的教育都讓我失望。",'], ['"21', 'cmn', '选择什么是“对”或“错”是一项艰难的任务，我们却必须要完成它。",'], ['"22', 'cmn', '這樣做的話什麼都不會改變的。",'], ['"23', 'cmn', '這個要三十歐元。",'], ['"24', 'cmn', '我一天賺一百歐元。",'], ['"25', 'cmn', '也许我会马上放弃然后去睡一觉。",'], ['"26', 'cmn', '那是不會發生的。",'], ['"27', 'cmn', '我会尽量不打扰你复习。",'], ['"28', 'cmn', '不要擔心。",'], ['"29', 'cmn', '我很想你。",'], ['"30', 'cmn', '我明天回來的時候會跟他們聯絡。",'], ['"3

Clearly for languages other than Chinese, there is an extra comma at the end
A decode 'utf'-8 needs to be used for the code to be able to process this mix of languages

In [7]:
#This is the code from the Map-Reduce
def mapper_parse_rows(row):
    try:
        data = row.decode('utf-8').split('\t')
        print(data)
            # Links Data
        if len(data) == 2:
            symbol = 'A'         
            orig_id = data[0]
            pp_id= data[1]
            yield pp_id, [symbol, orig_id]

            #Sentence Data
        else:
            symbol = 'B'
            pp_id = data[0]
            lang = data[1]
            text = data[2].strip().lower()                
            if lang == "en":
                yield pp_id, [symbol, text]
            else:
                pass
    except:
        pass

In [22]:
def mapper_parse_rows(row):
    data = row.split('\t')
    print(data)
    # Links Data
    if len(data) == 3:
        symbol = 'B'
        pp_id = data[0]
        lang = data[1]
        text = data[2].strip().lower()
        return pp_id, lang, text
    else:
        print("Didn't split!")
        pass
    


In [25]:
test_tsv = "cmn	æˆ‘å€‘è©¦è©¦çœ‹ï¼"
test_tsv_german = "184	deu	Wenn jemand fragen sollte"

In [26]:
print(mapper_parse_rows(test_tsv))
print(mapper_parse_rows(test_tsv_german))

['cmn', 'æˆ‘å€‘è©¦è©¦çœ‹ï¼\x81']
Didn't split!
None
['184', 'deu', 'Wenn jemand fragen sollte']
('184', 'deu', 'wenn jemand fragen sollte')


This error happened because encode is on strings and decodes is on bytes
https://stackoverflow.com/questions/26125141/str-object-has-no-attribute-decode-in-python3


In [29]:
english = pd.read_csv('eng_sentences.tsv', 
                           sep='\t', header=None, error_bad_lines=False)

In [31]:
english.shape

(1323161, 3)

In [3]:
len(sent_details)

8377731

In [ ]:
sent_details.dropna(subset=['Lang'], inplace=True)

In [ ]:
len(sent_details)

In [ ]:
langs = sorted(sent_details['Lang'].unique())

In [ ]:
required_langs = ['jap', 'en', 'hun', 'kor', 'vie', 'por']

In [43]:
sent_details.columns

Index(['Sentence id', 'Lang', 'Text', 'Username', 'Date added',
       'Date last modified'],
      dtype='object')

In [17]:
links.columns

Index(['Sentence id', 'Translation id'], dtype='object')

In [21]:
#merged_inner.columns

Index(['Sentence id', 'Lang', 'Text', 'Username', 'Date added',
       'Date last modified', 'Translation id'],
      dtype='object')

### The Core Merges Across Language Pairs

In [64]:
#For Spanish
t0 = time.time()
#approved_langs = ['eng', 'fra']
lang_raw = sent_details[(sent_details['Lang'] == 'eng')| (sent_details['Lang'] == 'spa')]
merged_inner = pd.merge(left=lang_raw, right=links, left_on='Sentence id', right_on='Sentence id')
final = pd.merge(left= merged_inner, right = lang_raw, left_on= 'Translation id', right_on = 'Sentence id')
final_en = final[(final['Lang_x']=='eng') & (final['Lang_y']!='eng')]
#This ensures that we don't simply find paraphrases of the sentence in the same language
final_pairs=final_en[['Text_x', 'Text_y']]
final_pairs= final_pairs.reset_index()
final_pairs.sort_values('Text_x')
final_pairs.sort_values('index')
print(("The time to run was {} seconds".format(time.time()-t0)))
#This took 272 seconds

The time to run was 214.40246987342834 seconds


In [25]:
final_pairs.head()

,index,Text_x,Text_y


In [63]:
# FOr French
t0 = time.time()
#approved_langs = ['eng', 'fra']
lang_raw = sent_details[(sent_details['Lang'] == 'eng')| (sent_details['Lang'] == 'fra')]
merged_inner = pd.merge(left=lang_raw, right=links, left_on='Sentence id', right_on='Sentence id')
final = pd.merge(left= merged_inner, right = lang_raw, left_on= 'Translation id', right_on = 'Sentence id')
final_en = final[(final['Lang_x']=='eng') & (final['Lang_y']!='eng')]
#This ensures that we don't simply find paraphrases of the sentence in the same language
final_pairs=final_en[['Text_x', 'Text_y']]
final_pairs= final_pairs.reset_index()
final_pairs.sort_values('Text_x')
final_pairs.sort_values('index')
print(("The time to run was {} seconds".format(time.time()-t0)))

The time to run was 393.89608001708984 seconds


In [26]:
#len(final_pairs.dropna(how='any'))

218861

Increase You Try to Recover

In [28]:
pickle.dump(final_pairs, open('tatoeba.pkl', 'wb'))

In [44]:
final_pairs = pd.read_pickle('tatoeba.pkl')


In [45]:
final_pairs

,index,Text_x,Text_y
0,0,Let's try something.,Vamos tentar alguma coisa!
1,2,Let's try something.,Vamos tentar algo!
2,4,Let's try something.,Vamos tentar algo.
3,5,Let's try something.,Vamos tentar alguma coisa.
4,6,I have to go to sleep.,Preciso ir dormir.
5,8,I need to go to sleep.,Preciso ir dormir.
6,10,I have to go to sleep.,Tenho que ir dormir.
7,13,I need to go to sleep.,Tenho que ir dormir.
8,16,I have to go to sleep.,Preciso dormir.
9,18,I have to sleep.,Preciso dormir.


In [51]:
final_pairs['both']= final_pairs['Text_x'] + "|" + final_pairs['Text_y']

In [52]:
final_pairs['both'].to_csv('both-tatoeba.txt', index=True, sep= ' ', header=None)

In [40]:
len(final_pairs)

218861

In [53]:
final_pairs['Text_x'].to_csv('src-tatoeba.txt', index=False, sep= ' ', header=None)
final_pairs['Text_y'].to_csv('tgt-tatoeba.txt', index=True, sep= ' ', header=None)

In [31]:
len(final_pairs['Text_y'])

218861

## Inefficient Approach to Matching (Don't Try This At Home)

In [ ]:
#This last section is just to laught at the inefficient code this user tried, and then gloat
#Do not actually run this unless your Robin Williams in Bicentennial Man. One is glad to be of service!
lol_tatoeba = []
loopers = links.iterrows()
for idx, row in tqdm(loopers): # For each row of the links,returns a series object
    src_idx, trg_idx = row['Sentence id'], row['Translation id']
    try:
        src = sent_details[sent_details['Sentence id']==src_idx][['Text', 'Lang']].iloc[0]
        trg = sent_details[sent_details['Sentence id']==trg_idx][['Text', 'Lang']].iloc[0]
    except:
        continue
    lol_tatoeba.append([src_idx, trg_idx, src['Lang'], trg['Lang'], src['Text'], trg['Text']])

df_tatoeba = pd.DataFrame(lol_tatoeba)
df_tatoeba.rename(columns={0:'SRC ID', 1:'TRG ID', 2:'SRC LANG', 3:'TRG LANG', 
                          4:'SRC', 5:'TRG'}, inplace=True)
df_tatoeba.to_csv('tatoeba-sentpairs.tsv', sep='\t')

# Renaming it for 
parallel_sents = df_tatoeba

8755653it [4:20:55, 667.71it/s]